In [124]:
%run functions.py
% run archive/silhouette_plotter.py

In [125]:
table = 'eth-tokens.test.clean_avg_with_balances_tokens'
label_table = 'eth-tokens.alldata.etherscan_labelcloud'

In [126]:
df, dflabel = load_data_from_bigquery(table, label_table)

In [127]:
results = data_pipeline(df)

In [128]:
# #trying out feature scaling to see if we get better separation
# df['incoming_std_time_btwn_txns'] = df['incoming_std_time_btwn_txns']/df['incoming_avg_time_btwn_txns']
# df['outgoing_std_time_btwn_txns'] = df['outgoing_std_time_btwn_txns']/df['outgoing_avg_time_btwn_txns']
# results = data_pipeline(df)

In [129]:
cl = cluster(results, n_clusters=8)

In [130]:
assign_cluster_to_data(df, dflabel, cl)

In [ ]:
tsne_results = calc_tsne(results, perplexity=50,n_iter=250)

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 35917 samples in 0.024s...
[t-SNE] Computed neighbors for 35917 samples in 11.181s...
[t-SNE] Computed conditional probabilities for sample 1000 / 35917
[t-SNE] Computed conditional probabilities for sample 2000 / 35917
[t-SNE] Computed conditional probabilities for sample 3000 / 35917
[t-SNE] Computed conditional probabilities for sample 4000 / 35917
[t-SNE] Computed conditional probabilities for sample 5000 / 35917
[t-SNE] Computed conditional probabilities for sample 6000 / 35917
[t-SNE] Computed conditional probabilities for sample 7000 / 35917
[t-SNE] Computed conditional probabilities for sample 8000 / 35917
[t-SNE] Computed conditional probabilities for sample 9000 / 35917
[t-SNE] Computed conditional probabilities for sample 10000 / 35917
[t-SNE] Computed conditional probabilities for sample 11000 / 35917
[t-SNE] Computed conditional probabilities for sample 12000 / 35917
[t-SNE] Computed conditional probabilities for s

In [ ]:
plot_tsne(cl, tsne_results)

In [ ]:
#highlighting clusters of interest
categs = ('Exchange', 'Mining', 'ICO Wallets')
colors = ('green', 'blue', 'orange')

d = {}
for c in categs:
    d[c] = find_category_of_cluster(cl,dflabel, category=c)

clusters= list(d.values())

In [ ]:
plot_tsne_with_labeled_clusters(tsne_results, cl, clusters, categs, colors)

In [ ]:
plot_tsne_with_labels(tsne_results,df, dflabel,categs,colors)

In [ ]:
#highlighting clusters of interest
categs = ('Exchange', 'Mining')#, 'ICO Wallets')
colors = ('green', 'blue')#, 'orange')

d = {}
for c in categs:
    d[c] = find_category_of_cluster(cl,dflabel, category=c)

clusters= list(d.values())

In [ ]:
#recluster with only clusters of interest - will help with separation

subcl, subresults, newdf = recluster(df, cl, clusters, n_clusters=3)

In [ ]:
sub_tsne_results = calc_tsne(subresults)

In [ ]:
assign_cluster_to_data(newdf, dflabel, subcl)

In [ ]:
#highlighting clusters of interest

d = {}
for c in categs:
    d[c] = find_category_of_cluster(subcl,dflabel, category=c)

subclusters= list(d.values())

In [ ]:
plot_all(sub_tsne_results,subcl,newdf,dflabel,subclusters,categs,colors )
